In [1]:
%reset -f

In [2]:
%reload_ext sql
%sql sqlite:///../ontology/tmp/merged.db

In [3]:
%sql SELECT name FROM sqlite_master WHERE type='table';

 * sqlite:///../ontology/tmp/merged.db
Done.


name
term_association
has_oio_synonym_statement
anonymous_expression
anonymous_class_expression
anonymous_property_expression
anonymous_individual_expression
owl_restriction
owl_complex_axiom
prefix
statements


In [11]:
%sql SELECT * FROM prefix WHERE prefix LIKE "OM%" OR prefix LIKE "OR%" ORDER BY prefix;

 * sqlite:///../ontology/tmp/merged.db
Done.


prefix,base
OMIABIS,http://purl.obolibrary.org/obo/OMIABIS_
OMIABIS,http://purl.obolibrary.org/obo/OMIABIS_
OMIABIS,http://purl.obolibrary.org/obo/OMIABIS_
OMIABIS,http://purl.obolibrary.org/obo/OMIABIS_
OMIT,http://purl.obolibrary.org/obo/OMIT_
OMIT,http://purl.obolibrary.org/obo/OMIT_
OMIT,http://purl.obolibrary.org/obo/OMIT_
OMIT,http://purl.obolibrary.org/obo/OMIT_
OMO,http://purl.obolibrary.org/obo/OMO_
OMO,http://purl.obolibrary.org/obo/OMO_


In [4]:
import pandas as pd
from os.path import join
import textdistance

def save_csv(query, fn):
    dir_name = "dataframes"
    df = pd.DataFrame(query)
    df.to_csv(join(dir_name,fn), sep='\t', index = False)
    return df

def add_distance(df, col_name, txt_dist_pkg):
    df\
    .insert(\
            len(df.columns),\
            col_name,\
            df.apply\
            (\
                lambda x: txt_dist_pkg\
                 (\
                    x.subject_value.lower(), x.object_value.lower()\
                 ),\
                axis=1,\
            )\
           )

In [5]:
query = %sql SELECT * FROM statements WHERE subject LIKE "%MONDO:%" AND object LIKE "%ICD10%"
mondo_2_icd_df = save_csv(query, "mondo_2_icd.tsv")
mondo_2_icd_df.head()


 * sqlite:///../ontology/tmp/merged.db
Done.


,stanza,subject,predicate,object,value,datatype,language
0,MONDO:0000087,MONDO:0000087,skos:narrowMatch,<https://icd.codes/icd10cm/Q04.3>,None,None,None
1,MONDO:0000088,MONDO:0000088,skos:exactMatch,<https://icd.codes/icd10cm/E30.1>,None,None,None
2,MONDO:0000088,MONDO:0000088,skos:exactMatch,ICD10CM:E30.1,None,None,None
3,MONDO:0000107,MONDO:0000107,skos:narrowMatch,<https://icd.codes/icd10cm/Q75.8>,None,None,None
4,MONDO:0000110,MONDO:0000110,skos:narrowMatch,<https://icd.codes/icd10cm/Q30.2>,None,None,None


In [6]:
# query = %sql SELECT * FROM statements WHERE subject LIKE "%MONDO:%" OR subject LIKE "%ICD10%" AND object LIKE "%MONDO:%" OR object LIKE "%ICD10%"
# mondo_icd_2_mondo_icd_df = save_csv(query, "mondo_icd_2_mondo_icd.tsv")
# mondo_icd_2_mondo_icd_df.head()
%sql SELECT * FROM statements \
WHERE subject LIKE "%MONDO:%" OR subject LIKE "%ICD10%" \
AND object LIKE "%MONDO:%" OR object LIKE "%ICD10%" LIMIT 10


 * sqlite:///../ontology/tmp/merged.db
Done.


stanza,subject,predicate,object,value,datatype,language
MONDO:0100332,MONDO:0100332,rdfs:label,None,disease has primary infectious agent,xsd:string,None
MONDO:0100332,MONDO:0100332,oio:id,None,MONDO:0100332,xsd:string,None
MONDO:0100332,MONDO:0100332,oio:created_by,None,https://orcid.org/0000-0001-5208-3432,xsd:string,None
MONDO:0100332,MONDO:0100332,rdfs:subPropertyOf,RO:0014001,None,None,None
MONDO:0100332,MONDO:0100332,rdf:type,owl:ObjectProperty,None,None,None
MONDO:0100333,MONDO:0100333,rdfs:label,None,disease caused by reactivation of latent infectious agent,xsd:string,None
MONDO:0100333,MONDO:0100333,oio:id,None,MONDO:0100333,xsd:string,None
MONDO:0100333,MONDO:0100333,oio:created_by,None,https://orcid.org/0000-0001-5208-3432,xsd:string,None
MONDO:0100333,MONDO:0100333,rdfs:subPropertyOf,RO:0014001,None,None,None
MONDO:0100333,MONDO:0100333,rdf:type,owl:ObjectProperty,None,None,None


In [7]:
# query = %sql SELECT * FROM statements WHERE (subject LIKE "%MONDO:%" OR subject LIKE "%icd10cm%") AND predicate IN ("rdfs:label", "oio:hasExactSynonym") LIMIT 10
# mondo_icd_label_exact_synonym_df = save_csv(query, "mondo_icd_label_exact_synonym.tsv")
# mondo_icd_label_exact_synonym_df.head()
%sql SELECT * FROM statements WHERE (subject LIKE "%MONDO:%" OR subject LIKE "%icd10cm%")\
    AND predicate IN ("rdfs:label", "oio:hasExactSynonym") LIMIT 5


 * sqlite:///../ontology/tmp/merged.db
Done.


stanza,subject,predicate,object,value,datatype,language
ICD10CM:A00-B99,ICD10CM:A00-B99,oio:hasExactSynonym,None,diseases generally recognized as communicable or transmissible,None,en
ICD10CM:A00.0,ICD10CM:A00.0,oio:hasExactSynonym,None,Classical cholera,None,en
ICD10CM:A00.1,ICD10CM:A00.1,oio:hasExactSynonym,None,Cholera eltor,None,en
ICD10CM:A01.0,ICD10CM:A01.0,oio:hasExactSynonym,None,Infection due to Salmonella typhi,None,en
ICD10CM:A01.02,ICD10CM:A01.02,oio:hasExactSynonym,None,Typhoid myocarditis,None,en


In [8]:
%sql SELECT * FROM statements WHERE (subject LIKE "%icd10cm%") AND predicate IN ("rdfs:label", "oio:hasExactSynonym") LIMIT 10


 * sqlite:///../ontology/tmp/merged.db
Done.


stanza,subject,predicate,object,value,datatype,language
ICD10CM:A00-B99,ICD10CM:A00-B99,oio:hasExactSynonym,None,diseases generally recognized as communicable or transmissible,None,en
ICD10CM:A00.0,ICD10CM:A00.0,oio:hasExactSynonym,None,Classical cholera,None,en
ICD10CM:A00.1,ICD10CM:A00.1,oio:hasExactSynonym,None,Cholera eltor,None,en
ICD10CM:A01.0,ICD10CM:A01.0,oio:hasExactSynonym,None,Infection due to Salmonella typhi,None,en
ICD10CM:A01.02,ICD10CM:A01.02,oio:hasExactSynonym,None,Typhoid myocarditis,None,en
ICD10CM:A01.02,ICD10CM:A01.02,oio:hasExactSynonym,None,Typhoid endocarditis,None,en
ICD10CM:A01.4,ICD10CM:A01.4,oio:hasExactSynonym,None,Infection due to Salmonella paratyphi NOS,None,en
ICD10CM:A02,ICD10CM:A02,oio:hasExactSynonym,None,infection or foodborne intoxication due to any Salmonella species other than S. typhi and S. paratyphi,None,en
ICD10CM:A02.0,ICD10CM:A02.0,oio:hasExactSynonym,None,Salmonellosis,None,en
ICD10CM:A02.25,ICD10CM:A02.25,oio:hasExactSynonym,None,Salmonella tubulo-interstitial nephropathy,None,en


In [9]:
%sql SELECT * FROM statements WHERE (subject == "ICD10CM:D35.2") LIMIT 10


 * sqlite:///../ontology/tmp/merged.db
Done.


stanza,subject,predicate,object,value,datatype,language
ICD10CM:D35.2,ICD10CM:D35.2,rdf:type,owl:Class,None,None,None
ICD10CM:D35.2,ICD10CM:D35.2,rdfs:label,None,Benign neoplasm of pituitary gland,None,en
ICD10CM:D35.2,ICD10CM:D35.2,rdfs:subClassOf,ICD10CM:D35,None,None,None


In [10]:
%sql SELECT * FROM statements WHERE (object == "ICD10CM:D35.2") LIMIT 10

 * sqlite:///../ontology/tmp/merged.db
Done.


stanza,subject,predicate,object,value,datatype,language
MONDO:0021439,MONDO:0021439,skos:exactMatch,ICD10CM:D35.2,None,None,None


In [11]:
%sql SELECT * FROM statements WHERE (subject == "MONDO:0017359")

 * sqlite:///../ontology/tmp/merged.db
Done.


stanza,subject,predicate,object,value,datatype,language
MONDO:0017359,MONDO:0017359,IAO:0000115,None,"A group of five inherited disorders caused by mutations in the AUH, DNAJC19, OPA3, and TAZ genes. The disorders are characterized by impairment in the function of mitochondria, resulting in the accumulation and excretion of 3-methylglutaconic acid, and the presence of 3-methylglutaric acid in the urine.",xsd:string,None
MONDO:0017359,MONDO:0017359,obo:mondo#should_conform_to,obo:mondo/patterns/OMIM_phenotypic_series.yaml,None,None,None
MONDO:0017359,MONDO:0017359,oio:hasDbXref,None,DOID:0060336,xsd:string,None
MONDO:0017359,MONDO:0017359,oio:hasDbXref,None,ICD10CM:E71.1,xsd:string,None
MONDO:0017359,MONDO:0017359,oio:hasDbXref,None,ICD10CM:E71.111,xsd:string,None
MONDO:0017359,MONDO:0017359,oio:hasDbXref,None,MESH:C579867,xsd:string,None
MONDO:0017359,MONDO:0017359,oio:hasDbXref,None,NCIT:C98678,xsd:string,None
MONDO:0017359,MONDO:0017359,oio:hasDbXref,None,OMIMPS:250950,xsd:string,None
MONDO:0017359,MONDO:0017359,oio:hasDbXref,None,Orphanet:289902,xsd:string,None
MONDO:0017359,MONDO:0017359,oio:hasDbXref,None,SCTID:237950009,xsd:string,None


In [12]:
%sql SELECT * FROM entailed_edge WHERE object LIKE "%ICD10%" LIMIT 10

 * sqlite:///../ontology/tmp/merged.db
Done.


subject,predicate,object
ICD10CM:M61.052,rdfs:subClassOf,ICD10CM:M60-M63
ICD10CM:M61.052,rdfs:subClassOf,ICD10CM:M61.05
ICD10CM:M61.052,rdfs:subClassOf,ICD10CM:M61.0
ICD10CM:M61.052,rdfs:subClassOf,ICD10CM:M00-M99
ICD10CM:M61.052,rdfs:subClassOf,ICD10CM:M61.052
ICD10CM:M61.052,rdfs:subClassOf,ICD10CM:M61
ICD10WHO:M12.2,rdfs:subClassOf,ICD10WHO:M12.2
ICD10WHO:M12.2,rdfs:subClassOf,ICD10WHO:M00-M25
ICD10WHO:M12.2,rdfs:subClassOf,ICD10WHO:XIII
ICD10WHO:M12.2,rdfs:subClassOf,ICD10WHO:M12


In [15]:
%sql SELECT * FROM entailed_edge WHERE subject LIKE "%MONDO%" AND object LIKE "%ICD10%" LIMIT 10

 * sqlite:///../ontology/tmp/merged.db
Done.


subject,predicate,object


In [16]:
%sql SELECT * FROM entailed_edge WHERE subject LIKE "%ICD10%" AND object LIKE "%MONDO%"  LIMIT 10

 * sqlite:///../ontology/tmp/merged.db
Done.


subject,predicate,object


In [17]:
%sql SELECT * FROM entailed_edge WHERE subject LIKE "%ICD10%" LIMIT 10

 * sqlite:///../ontology/tmp/merged.db
Done.


subject,predicate,object
ICD10CM:M61.052,rdfs:subClassOf,ICD10CM:M60-M63
ICD10CM:M61.052,rdfs:subClassOf,ICD10CM:M61.05
ICD10CM:M61.052,rdfs:subClassOf,ICD10CM:M61.0
ICD10CM:M61.052,rdfs:subClassOf,ICD10CM:M00-M99
ICD10CM:M61.052,rdfs:subClassOf,ICD10CM:M61.052
ICD10CM:M61.052,rdfs:subClassOf,ICD10CM:M61
ICD10WHO:M12.2,rdfs:subClassOf,ICD10WHO:M12.2
ICD10WHO:M12.2,rdfs:subClassOf,ICD10WHO:M00-M25
ICD10WHO:M12.2,rdfs:subClassOf,ICD10WHO:XIII
ICD10WHO:M12.2,rdfs:subClassOf,ICD10WHO:M12


In [18]:
%sql SELECT * FROM rdfs_label_statement LIMIT 10

 * sqlite:///../ontology/tmp/merged.db
Done.


stanza,subject,predicate,object,value,datatype,language
IAO:0000111,IAO:0000111,rdfs:label,None,editor preferred term,None,en
IAO:0000112,IAO:0000112,rdfs:label,None,example of usage,None,en
IAO:0000113,IAO:0000113,rdfs:label,None,in branch,None,en
IAO:0000114,IAO:0000114,rdfs:label,None,has curation status,None,en
IAO:0000115,IAO:0000115,rdfs:label,None,definition,xsd:string,None
IAO:0000115,IAO:0000115,rdfs:label,None,definition,None,en
IAO:0000116,IAO:0000116,rdfs:label,None,editor note,None,en
IAO:0000117,IAO:0000117,rdfs:label,None,term editor,None,en
IAO:0000118,IAO:0000118,rdfs:label,None,alternative term,None,en
IAO:0000118,IAO:0000118,rdfs:label,None,alternative label,None,en


## Post-process some DataFrames

In [19]:
full_table_query = %sql SELECT subject, predicate, object, value, datatype FROM rdfs_label_statement
full_table_df = pd.DataFrame(full_table_query)
full_table_df.head()

 * sqlite:///../ontology/tmp/merged.db
Done.


,subject,predicate,object,value,datatype
0,IAO:0000111,rdfs:label,None,editor preferred term,None
1,IAO:0000112,rdfs:label,None,example of usage,None
2,IAO:0000113,rdfs:label,None,in branch,None
3,IAO:0000114,rdfs:label,None,has curation status,None
4,IAO:0000115,rdfs:label,None,definition,xsd:string


In [20]:
full_table_df_with_values = full_table_df[full_table_df['value'].values != None]
subject_value_df = full_table_df_with_values[['subject', "value"]]
subject_value_df.head()

,subject,value
0,IAO:0000111,editor preferred term
1,IAO:0000112,example of usage
2,IAO:0000113,in branch
3,IAO:0000114,has curation status
4,IAO:0000115,definition


In [21]:
cols_to_drop = ['stanza','value', 'datatype','language']
mondo_2_icd_df_subset = mondo_2_icd_df.drop(columns=cols_to_drop)
mondo_2_icd_df_subset.head()

,subject,predicate,object
0,MONDO:0000087,skos:narrowMatch,<https://icd.codes/icd10cm/Q04.3>
1,MONDO:0000088,skos:exactMatch,<https://icd.codes/icd10cm/E30.1>
2,MONDO:0000088,skos:exactMatch,ICD10CM:E30.1
3,MONDO:0000107,skos:narrowMatch,<https://icd.codes/icd10cm/Q75.8>
4,MONDO:0000110,skos:narrowMatch,<https://icd.codes/icd10cm/Q30.2>


In [22]:
mondo_2_icd_subset_with_values = mondo_2_icd_df_subset\
                                .merge(subject_value_df, how="left", on="subject")\
                                .rename(columns={'value': "subject_value"})
mondo_2_icd_subset_with_values.head()


,subject,predicate,object,subject_value
0,MONDO:0000087,skos:narrowMatch,<https://icd.codes/icd10cm/Q04.3>,polymicrogyria
1,MONDO:0000088,skos:exactMatch,<https://icd.codes/icd10cm/E30.1>,precocious puberty
2,MONDO:0000088,skos:exactMatch,ICD10CM:E30.1,precocious puberty
3,MONDO:0000107,skos:narrowMatch,<https://icd.codes/icd10cm/Q75.8>,auriculocondylar syndrome
4,MONDO:0000110,skos:narrowMatch,<https://icd.codes/icd10cm/Q30.2>,bifid nose


In [23]:
mondo_2_icd_values_df = mondo_2_icd_subset_with_values\
                        .merge(subject_value_df, how="left", left_on="object", right_on="subject")\
                        .rename(columns={'value': "object_value", "subject_x": "subject"})\
                        .drop(columns=['subject_y'])\
                        .fillna("")
        

mondo_2_icd_values_df.head()


,subject,predicate,object,subject_value,object_value
0,MONDO:0000087,skos:narrowMatch,<https://icd.codes/icd10cm/Q04.3>,polymicrogyria,
1,MONDO:0000088,skos:exactMatch,<https://icd.codes/icd10cm/E30.1>,precocious puberty,
2,MONDO:0000088,skos:exactMatch,ICD10CM:E30.1,precocious puberty,Precocious puberty
3,MONDO:0000107,skos:narrowMatch,<https://icd.codes/icd10cm/Q75.8>,auriculocondylar syndrome,
4,MONDO:0000110,skos:narrowMatch,<https://icd.codes/icd10cm/Q30.2>,bifid nose,


In [24]:
mondo_2_icd_values_df = mondo_2_icd_values_df[['subject', 'subject_value', 'predicate', 'object', 'object_value' ]]
mondo_2_icd_values_df.to_csv(join("dataframes","mondo2icd_with_values.tsv"), sep='\t', index=False)


## Subject and Object value text-distances

In [25]:
# Add Levenshtein distance [ 0: Perfect match]
add_distance(mondo_2_icd_values_df, "levenshtein_dist", textdistance.levenshtein.distance)

In [26]:
# Add Jaccard Index [ 0: Perfect match]
add_distance(mondo_2_icd_values_df, "jaccard_index", textdistance.jaccard.distance)

In [27]:
# Add Monge-Elkan Distance [ 0: Perfect match]
add_distance(mondo_2_icd_values_df, "monge_elkan", textdistance.monge_elkan.distance)
mondo_2_icd_values_df.to_csv(join("dataframes","mondo2icd_with_distances.tsv"), sep='\t', index=False)
mondo_2_icd_values_df.head()

,subject,subject_value,predicate,object,object_value,levenshtein_dist,jaccard_index,monge_elkan
0,MONDO:0000087,polymicrogyria,skos:narrowMatch,<https://icd.codes/icd10cm/Q04.3>,,14,1.0,2.0
1,MONDO:0000088,precocious puberty,skos:exactMatch,<https://icd.codes/icd10cm/E30.1>,,18,1.0,2.0
2,MONDO:0000088,precocious puberty,skos:exactMatch,ICD10CM:E30.1,Precocious puberty,0,0.0,0.0
3,MONDO:0000107,auriculocondylar syndrome,skos:narrowMatch,<https://icd.codes/icd10cm/Q75.8>,,25,1.0,2.0
4,MONDO:0000110,bifid nose,skos:narrowMatch,<https://icd.codes/icd10cm/Q30.2>,,10,1.0,2.0


In [28]:
condition_1 = mondo_2_icd_values_df['levenshtein_dist'] < 10
condition_2 = mondo_2_icd_values_df['predicate'] != 'skos:exactMatch'
condition_3 = mondo_2_icd_values_df['object_value'] != ""

mondo_2_icd_values_df.loc[(condition_1) & (condition_2) & (condition_3)]

,subject,subject_value,predicate,object,object_value,levenshtein_dist,jaccard_index,monge_elkan
77,MONDO:0000827,salmonellosis,skos:broadMatch,ICD10CM:A02.1,Salmonella sepsis,5,0.333333,1.923077
78,MONDO:0000827,salmonellosis,skos:broadMatch,ICD10CM:A02.0,Salmonella enteritis,9,0.500000,1.923077
207,MONDO:0001195,spotted fever,skos:broadMatch,ICD10CM:A77.8,Other spotted fevers,7,0.350000,1.923077
1171,MONDO:0005055,Kaposi's sarcoma,skos:broadMatch,ICD10CM:C46.0,Kaposi's sarcoma of skin,8,0.333333,1.937500
1205,MONDO:0005081,preeclampsia,skos:broadMatch,ICD10CM:O14.1,Severe pre-eclampsia,8,0.400000,1.916667
1402,MONDO:0005504,diphtheria,skos:broadMatch,ICD10CM:A36.8,Other diphtheria,6,0.375000,1.900000
1412,MONDO:0005526,tetanus,skos:broadMatch,ICD10CM:A35,Other tetanus,6,0.461538,1.857143
1435,MONDO:0005615,plasmacytoma,skos:broadMatch,ICD10CM:C90.3,Solitary plasmacytoma,9,0.428571,1.916667
1443,MONDO:0005631,actinomycosis,skos:broadMatch,ICD10CM:A42.7,Actinomycotic sepsis,7,0.350000,1.923077
1453,MONDO:0005645,ancylostomiasis,skos:broadMatch,ICD10CM:B76.0,Ancylostomiasis,0,0.000000,0.000000
